In [9]:
import pandas as pd
import numpy as np
import datetime as dt
import requests

from sys import platform

if platform == "win32":
    path = 'C:/Users/olive/GitHub/f1-analytics/'
elif platform == "darwin":
    path = '/Users/oliverjcarter/Documents/GitHub/f1-analytics/'

In [10]:
races = pd.read_csv(path+'data/merged.csv')

# weather = races.iloc[:,[0,1,2,3,4,5,6]]
races.head()

,season,round,circuit_id,country,lat,long,date,driver,date_of_birth,nationality,...,driver_standings_pos,constructor_points,constructor_wins,constructor_standings_pos,pos,final_time,q_delta,q1,q2,q3
0,2014,1,albert_park,Australia,-37.8497,144.968,2014-03-16,rosberg,1985-06-27,German,...,1,25,1,2,3,104.595,0.364,92.564,102.264,104.595
1,2014,1,albert_park,Australia,-37.8497,144.968,2014-03-16,kevin_magnussen,1992-10-05,Danish,...,2,33,0,1,4,105.745,1.514,90.949,103.247,105.745
2,2014,1,albert_park,Australia,-37.8497,144.968,2014-03-16,alonso,1981-07-29,Spanish,...,4,18,0,3,5,105.819,1.588,91.388,102.805,105.819
3,2014,1,albert_park,Australia,-37.8497,144.968,2014-03-16,bottas,1989-08-28,Finnish,...,5,10,0,4,10,108.147,3.916,91.601,103.852,108.147
4,2014,1,albert_park,Australia,-37.8497,144.968,2014-03-16,hulkenberg,1987-08-19,German,...,6,9,0,5,7,106.030,1.799,93.893,103.658,106.03


In [11]:
base = 'http://api.openweathermap.org/data/2.5/weather?'
api_key = open(path+'api_key.txt', 'r').read()
city = 'Cardiff'

url =  base + 'appid=' + api_key + '&q=' + city
response = requests.get(url)
print(response)

<Response [200]>
